In [20]:
import pandas as pd
import os
from glob import glob

In [133]:
historic_hu = pd.read_csv('data/historic/HB_HOUSTON.csv')
live_hu = pd.read_csv('data/live/HB_HOUSTON.csv')

In [203]:
def fix_data(df):
    '''
    Convert DeliveryDate, DeliveryHour and DeliveryInterval column to pd.to_datetime. Also remove the DSTFlag and SettlementPointType column. 
    Also arranges in ascending order
    '''
    df['Date'] = pd.to_datetime(df['DeliveryDate']) + df['DeliveryHour'].astype('timedelta64[h]') + ((df['DeliveryInterval'] - 1) * 15).astype('timedelta64[m]')
    df = df.drop(columns=['DeliveryDate', 'DeliveryHour', 'DeliveryInterval', 'DSTFlag', 'SettlementPointType', 'SettlementPointName'], axis=1)
    df = df.sort_values('Date').reset_index(drop=True)

    return df[['Date', 'SettlementPointPrice']]

In [204]:
live = fix_data(live_hu)
historic = fix_data(historic_hu)

In [279]:
def fixJoin(live, historic):
    '''
    Drop Duplicates and fix missing timelines
    '''
    
    df = pd.concat([live, historic]).sort_values('Date')
    dupShape = df.shape[0]

    df = df.drop_duplicates('Date').reset_index(drop=True)
    duplicates = dupShape - df.shape[0]
    print("Total Duplicates: {}".format(duplicates))
    
    totalMissing = sum((df['Date'].shift(-1)[:-1] - df['Date'][:-1]).astype('timedelta64[m]') != 15)
    
    print("Total Missing Dates: {}".format(totalMissing))
    missingDates = list(df[:-1][(df['Date'].shift(-1)[:-1] - df['Date'][:-1]).astype('timedelta64[m]') != 15]['Date'])
    print("The Missing dates are around: {}".format(missingDates))
    
    dates = pd.DataFrame(pd.date_range(df.iloc[0]['Date'],df.iloc[-1]['Date'],freq='15T'))
    dates.columns = ['Date']
    
    df.set_index('Date', inplace=True)
    dates.set_index('Date', inplace=True)
    
    full_data = pd.concat([df, dates], axis=1).fillna(method='ffill')
    full_data.index.name = 'Date'
    
    return full_data.reset_index()

In [280]:
df = fixJoin(live, historic)

Total Duplicates: 22393
Total Missing Dates: 8
The Missing dates are around: [Timestamp('2011-03-13 02:45:00'), Timestamp('2012-03-11 02:45:00'), Timestamp('2013-03-10 02:45:00'), Timestamp('2014-03-09 02:45:00'), Timestamp('2015-03-08 02:45:00'), Timestamp('2016-03-13 02:45:00'), Timestamp('2017-03-12 02:45:00'), Timestamp('2018-03-11 02:45:00')]


In [285]:
df

,Date,SettlementPointPrice
0,2010-12-01 01:00:00,25.08
1,2010-12-01 01:15:00,23.20
2,2010-12-01 01:30:00,23.45
3,2010-12-01 01:45:00,22.17
4,2010-12-01 02:00:00,21.96
5,2010-12-01 02:15:00,21.58
6,2010-12-01 02:30:00,21.34
7,2010-12-01 02:45:00,20.99
8,2010-12-01 03:00:00,21.25
9,2010-12-01 03:15:00,21.54
